In [23]:
import pyomo.environ as pyo
import numpy as np
import matplotlib.pyplot as plt

# Classic Route Planning Problem:

![](../01_Introduction/RoutePlanningProblem.png)

|$r$|$\delta_r$|
|-|-|
|$AB$|15|
|$AC$|5|
|$BC$|4|
|$BD$|2|
|$CD$|10|

$$\min_{X_r,Y_p} \sum_{r \in \textbf{R}} \delta_r X_r$$
$$s.t.\ \ \ \sum_{r \in \textbf{R}_p} X_r = 2 Y_p \ \ \ \forall p \in \textbf{P}^{NON-TERM}$$
$$\sum_{r \in \textbf{R}_p} X_r = Y_p \ \ \ \forall p \in \textbf{P}^{TERM}$$
$$Y_p = 1 \ \ \ \forall p \in \textbf{P}^{TERM}$$
$$X_r, Y_p \in \{0,1\}$$

In [24]:
class Parameters:
    """
    A class to house all the parameters relevant to the Route Planning Example Problem
    """
    def __init__(self):
        #Specify all parameters here making sure to say "self." before each one.
        self.Points = ["A","B","C","D"]
        self.Connections = ["AB","AC","BC","BD","CD"]
        self.TerminalPoints = ["A","D"]

        self.delta = {
            "AB": 15,
            "AC": 5,
            "BC": 4,
            "BD": 2,
            "CD": 10
        }

In [25]:
from PyomoTools import LoadIndexedSet

def AssembleModel(params:Parameters):
    """
    A function to generate an instance of the Pyomo model for the Route Planning Example Problem.

    Parameters
    ----------
    params: Parameters
        The Parameters object containing the parameters relevant to this instance.

    Returns
    -------
    model: pyo.ConcreteModel
        The Pyomo model for this instance.
    """

    ### STEP 1: Define the Pyomo model
    model = pyo.ConcreteModel()

    ### STEP 2: Define sets
    model.P = pyo.Set(initialize=params.Points)
    model.P_NON_TERM = pyo.Set(initialize=list(set(params.Points) - set(params.TerminalPoints)))
    model.P_TERM = pyo.Set(initialize=params.TerminalPoints)
    model.R = pyo.Set(initialize=params.Connections)
    
    Rp = {p: [r for r in model.R if p in r] for p in model.P}
    LoadIndexedSet(model,"R_p",Rp) #This is a function I created to automate the process of adding subsets into a Pyomo model.

    ### Step 3: Define oarameters
    #This is already done in the Parameters class

    ### Step 4: Define variables
    model.X = pyo.Var(model.R,domain=pyo.Binary)

    model.Y = pyo.Var(model.P,domain=pyo.Binary)

    ### Step 5: Define constraints
    def NonTerminalTravelConstraint(model,p):
        return sum(model.X[r] for r in model.R_p[p]) == 2 * model.Y[p]
    model.NonTerminalTravelConstraint = pyo.Constraint(model.P_NON_TERM,rule=NonTerminalTravelConstraint)

    def TerminalTravelConstraint(model,p):
        return sum(model.X[r] for r in model.R_p[p]) == model.Y[p]
    model.TerminalTravelConstraint = pyo.Constraint(model.P_TERM,rule=TerminalTravelConstraint)

    def TerminalConstraint(model,p):
        return model.Y[p] == 1
    model.TerminalConstraint = pyo.Constraint(model.P_TERM,rule=TerminalConstraint)

    ### Step 6: Define Objective
    model.Obj = pyo.Objective(expr=sum(params.delta[r] * model.X[r] for r in model.R))

    return model


In [26]:
def ExecuteOptimization(model:pyo.ConcreteModel):
    """
    A function to execute the optimization of a pyomo model.

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model you'd like to optimize

    Returns
    -------
    None
    """

    #Sometimes there are lots of settings you want to set or special functions you want to call to record the output of this "solve call". That's why I normally have a dedicated function for this. But I suppose it's less necessary for this simple problem.
    solver = pyo.SolverFactory("scip")
    solver.solve(model)

In [27]:
def ExtractResults(model:pyo.ConcreteModel):
    """
    A function to extract the results of a Route Planning Problem model

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model from which you'd like to extract the results

    Returns
    -------
    traveledPaths: list
        A list of the paths that are to be traveled under the optimal solution
    """
    traveledPaths = []
    for r in model.R:
        Xval = pyo.value(model.X[r])
        if Xval >= 0.5:
            traveledPaths.append(r)
    return traveledPaths

In [28]:
def main():
    params = Parameters()
    model = AssembleModel(params)
    ExecuteOptimization(model)
    results = ExtractResults(model)

    print(results)

main()

['AC', 'BC', 'BD']


# Route Planning Problem with Multiple Time Periods

![](RoutePlanningProblemRepeated.png)

$\delta_{r,t}$

|$r$|$t=0$|$t=1$|$t=2$|$t=3$|
|-|-|-|-|-|
|$AB$|$15|$10|$8|$8|
|$AC$|$5|$5|$9|$10|
|$BC$|$4|$10|$13|$13|
|$BD$|$2|$2|$6|$5|
|$CD$|$10|$15|$4|$7|

$$\min_{X_{r,t},Y_{r,t}} \sum_{r \in \textbf{R}} \sum_{t \in \textbf{T}} \delta_{r,t} X_{r,t}$$
$$s.t.\ \ \ Y_{p,t+1} - Y_{p,t} = \sum_{r\in\textbf{R}^+_p} X_{r,t} - \sum_{r\in\textbf{R}^-_p} X_{r,t}$$
$$\phantom\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall p \in \textbf{P}, t \in \textbf{T} \neq 3$$
$$\sum_{r \in \textbf{R}} X_{r,t} \leq 1 \ \ \ \forall t \in \textbf{T}$$
$$\sum_{p \in \textbf{P}} Y_{p,t} = 1 \ \ \ \forall t \in \textbf{T}$$
$$Y_{A,0} = 1$$
$$Y_{3,D} = 1$$
$$X_{r,t}, Y_{p,t} \in \{0,1\}$$

In [29]:
class Parameters:
    """
    A class to house all the parameters relevant to the Route Planning with Multiple Time Periods Example Problem
    """
    def __init__(self):
        #Specify all parameters here making sure to say "self." before each one.
        self.Points = ["A","B","C","D"]
        self.Connections = ["AB","AC","BC","BD","CD"]
        self.StartingPoint = "A"
        self.EndingPoint = "D"

        self.TimePoints = [0,1,2,3]

        self.delta = {
            "AB": {0: 15., 1: 10., 2: 8. , 3: 8. },
            "AC": {0: 5. , 1: 5. , 2: 9. , 3: 10.},
            "BC": {0: 4. , 1: 10., 2: 13., 3: 13.},
            "BD": {0: 2. , 1: 2. , 2: 6. , 3: 5. },
            "CD": {0: 10., 1: 15., 2: 4. , 3: 7. }
        }

In [30]:
from PyomoTools import LoadIndexedSet

def AssembleModel(params:Parameters):
    """
    A function to generate an instance of the Pyomo model for the Route Planning with Multiple Time Periods Example Problem.

    Parameters
    ----------
    params: Parameters
        The Parameters object containing the parameters relevant to this instance.

    Returns
    -------
    model: pyo.ConcreteModel
        The Pyomo model for this instance.
    """

    ### STEP 1: Define the Pyomo model
    model = pyo.ConcreteModel()

    ### STEP 2: Define sets
    model.P = pyo.Set(initialize=params.Points)
    model.R = pyo.Set(initialize=params.Connections)
    
    R_plus = {p: [r for r in model.R if p == r[1]] for p in model.P}
    R_minus = {p: [r for r in model.R if p == r[0]] for p in model.P}
    LoadIndexedSet(model,"R_plus",R_plus) #This is a function I created to automate the process of adding subsets into a Pyomo model.
    LoadIndexedSet(model,"R_minus",R_minus)
    
    model.T = pyo.Set(initialize=params.TimePoints)
    tInit = min(params.TimePoints)
    tEnd = max(params.TimePoints)
    model.T_NON_END = pyo.Set(initialize=[t for t in model.T if t != tEnd])

    ### Step 3: Define parameters
    #This is already done in the Parameters class

    ### Step 4: Define variables
    model.X = pyo.Var(model.R * model.T,domain=pyo.Binary)

    model.Y = pyo.Var(model.P * model.T,domain=pyo.Binary)

    ### Step 5: Define constraints
    def TransitionDefinitionConstraint(model,p,t):
        return model.Y[p,t+1] - model.Y[p,t] == sum(model.X[r,t] for r in model.R_plus[p]) - sum(model.X[r,t] for r in model.R_minus[p])
    model.TransitionDefinitionConstraint = pyo.Constraint(model.P * model.T_NON_END,rule=TransitionDefinitionConstraint)

    def NoMoreThanOneTransitionAtATime(model,t):
        return sum(model.X[r,t] for r in model.R) <= 1
    model.NoMoreThanOneTransitionAtATime = pyo.Constraint(model.T, rule=NoMoreThanOneTransitionAtATime)

    def OnePositionAtATime(model,t):
        return sum(model.Y[p,t] for p in model.P) == 1
    model.OnePositionAtATime = pyo.Constraint(model.T, rule=OnePositionAtATime)

    def InitialPositionConstraint(model):
        return model.Y[params.StartingPoint,tInit] == 1
    model.InitialPositionConstraint = pyo.Constraint(rule=InitialPositionConstraint)

    def FinalPositionConstraint(model):
        return model.Y[params.EndingPoint,tEnd] == 1
    model.FinalPositionConstraint = pyo.Constraint(rule=FinalPositionConstraint)

    ### Step 6: Define Objective
    model.Obj = pyo.Objective(expr=sum(params.delta[r][t] * model.X[r,t] for r in model.R for t in model.T),sense=pyo.minimize)

    return model

In [31]:
def ExecuteOptimization(model:pyo.ConcreteModel,tee=False):
    """
    A function to execute the optimization of a pyomo model.

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model you'd like to optimize
    tee: boolean (optional, Default=False)
        An indication of whether or not you'd like to print the solver's progress as it solves.

    Returns
    -------
    None
    """

    #Sometimes there are lots of settings you want to set or special functions you want to call to record the output of this "solve call". That's why I normally have a dedicated function for this. But I suppose it's less necessary for this simple problem.
    solver = pyo.SolverFactory("scip")
    solver.solve(model,tee=tee)

In [32]:
def ExtractResults(model:pyo.ConcreteModel):
    """
    A function to extract the results of a Route Planning Problem with Multiple Time Periods model

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model from which you'd like to extract the results

    Returns
    -------
    traveledPaths: list
        A list of the paths that are to be traveled under the optimal solution
    """
    instructions = []
    for t in model.T:
        for r in model.R:
            Xval = pyo.value(model.X[r,t])
            if Xval >= 0.5:
                instructions.append(f"At t = {t}, travel down path {r}")
    return instructions

In [33]:
def main():
    params = Parameters()
    model = AssembleModel(params)
    ExecuteOptimization(model)
    results = ExtractResults(model)

    for step in results:
        print(step)

main()

At t = 0, travel down path AC
At t = 2, travel down path CD


# Route Planning Problem with Different Scenarios

![](RoutePlanningProblemWithScenarios.png)

$\delta_{r,t,s_1}$

|$r$|$t=0$|$t=1$|$t=2$|$t=3$|
|-|-|-|-|-|
|$AB$|$5|$10|$12|$9|
|$AC$|$30|$10|$12|$13|
|$BC$|$4|$5|$6|$7|
|$BD$|$2|$25|$50|$2|
|$CD$|$10|$12|$13|$15|

$\delta_{r,t,s_2}$

|$r$|$t=0$|$t=1$|$t=2$|$t=3$|
|-|-|-|-|-|
|$AB$|$5|$8|$10|$7|
|$AC$|$30|$5|$9|$10|
|$BC$|$4|$10|$13|$13|
|$BD$|$2|$2|$6|$6|
|$CD$|$10|$15|$4|$10|

$$\min_{X_{r,t,s},Y_{r,t,s}} \sum_{s \in \textbf{S}} \pi_s \left( \sum_{r \in \textbf{R}} \sum_{t \in \textbf{T}} \delta_{r,t,s} X_{r,t,s}\right)$$
$$s.t.\ \ \ Y_{p,t+1,s} - Y_{p,t,s} = \sum_{r\in\textbf{R}^+_p} X_{r,t,s} - \sum_{r\in\textbf{R}^-_p} X_{r,t,s}$$
$$\phantom\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall p \in \textbf{P}, t \in \textbf{T} \neq 3, s \in \textbf{S}$$
$$\sum_{r \in \textbf{R}} X_{r,t,s} \leq 1 \ \ \ \forall t \in \textbf{T}, s \in \textbf{S}$$
$$\sum_{p \in \textbf{P}} Y_{p,t,s} = 1 \ \ \ \forall t \in \textbf{T}, s \in \textbf{S}$$
$$Y_{A,0,s} = 1 \ \ \ \forall s \in \textbf{S}$$
$$Y_{3,D,s} = 1 \ \ \ \forall s \in \textbf{S}$$

$$X_{r,0,s_1} = X_{r,0,s} \ \ \ \forall r \in \textbf{R}, s \in \textbf{S} \neq s_1$$
$$Y_{p,0,s_1} = Y_{p,0,s} \ \ \ \forall p \in \textbf{P}, s \in \textbf{S} \neq s_1$$

$$X_{r,t,s}, Y_{p,t,s} \in \{0,1\}$$

In [34]:
class Parameters:
    """
    A class to house all the parameters relevant to the Route Planning with Multiple Time Periods & Multiple Scenarios Example Problem
    """
    def __init__(self):
        #Specify all parameters here making sure to say "self." before each one.
        self.Points = ["A","B","C","D"]
        self.Connections = ["AB","AC","BC","BD","CD"]
        self.StartingPoint = "A"
        self.EndingPoint = "D"

        self.TimePoints = [0,1,2,3]
        self.Scenarios = [1,2]

        #Indexed by "r" then "t" then "s"
        self.delta = {
            "AB": {
                0: {1:  5, 2:  5}, 
                1: {1: 10, 2:  8}, 
                2: {1: 12, 2: 10},
                3: {1:  9, 2:  7}
            },
            "AC": {
                0: {1: 30, 2:  30},
                1: {1: 10, 2:  5},
                2: {1: 12, 2:  9},
                3: {1: 13, 2: 10}
            },
            "BC": {
                0: {1:  4, 2:  4},
                1: {1:  5, 2: 10},
                2: {1:  6, 2: 13},
                3: {1:  7, 2: 13}
            },
            "BD": {
                0: {1:  2, 2:  2},
                1: {1: 25, 2:  2},
                2: {1: 50, 2:  6},
                3: {1:  2, 2:  6}
            },
            "CD": {
                0: {1: 10, 2: 10}, 
                1: {1: 12, 2: 15}, 
                2: {1: 13, 2:  4},
                3: {1: 15, 2: 10}
            }
        }

        self.pi = {
            1: 0.75,
            2: 0.25
        }

In [35]:
from PyomoTools import LoadIndexedSet, InfeasibilityReport

def AssembleModel(params:Parameters):
    """
    A function to generate an instance of the Pyomo model for the Route Planning with Multiple Time Periods Example Problem.

    Parameters
    ----------
    params: Parameters
        The Parameters object containing the parameters relevant to this instance.

    Returns
    -------
    model: pyo.ConcreteModel
        The Pyomo model for this instance.
    """

    ### STEP 1: Define the Pyomo model
    model = pyo.ConcreteModel()

    ### STEP 2: Define sets
    model.P = pyo.Set(initialize=params.Points)
    model.R = pyo.Set(initialize=params.Connections)
    
    R_plus = {p: [r for r in model.R if p == r[1]] for p in model.P}
    R_minus = {p: [r for r in model.R if p == r[0]] for p in model.P}
    LoadIndexedSet(model,"R_plus",R_plus) #This is a function I created to automate the process of adding subsets into a Pyomo model.
    LoadIndexedSet(model,"R_minus",R_minus)
    
    model.T = pyo.Set(initialize=params.TimePoints)
    tInit = min(params.TimePoints)
    tEnd = max(params.TimePoints)
    model.T_NON_END = pyo.Set(initialize=[t for t in model.T if t != tEnd])

    model.S = pyo.Set(initialize=params.Scenarios)
    model.S_NON_1 = pyo.Set(initialize=[s for s in params.Scenarios if s != 1])

    ### Step 3: Define parameters
    #This is already done in the Parameters class

    ### Step 4: Define variables
    model.X = pyo.Var(model.R * model.T * model.S,domain=pyo.Binary)

    model.Y = pyo.Var(model.P * model.T * model.S,domain=pyo.Binary)

    ### Step 5: Define constraints
    def TransitionDefinitionConstraint(model,p,t,s):
        return model.Y[p,t+1,s] - model.Y[p,t,s] == sum(model.X[r,t,s] for r in model.R_plus[p]) - sum(model.X[r,t,s] for r in model.R_minus[p])
    model.TransitionDefinitionConstraint = pyo.Constraint(model.P * model.T_NON_END * model.S,rule=TransitionDefinitionConstraint)

    def NoMoreThanOneTransitionAtATime(model,t,s):
        return sum(model.X[r,t,s] for r in model.R) <= 1
    model.NoMoreThanOneTransitionAtATime = pyo.Constraint(model.T * model.S, rule=NoMoreThanOneTransitionAtATime)

    def OnePositionAtATime(model,t,s):
        return sum(model.Y[p,t,s] for p in model.P) == 1
    model.OnePositionAtATime = pyo.Constraint(model.T*model.S, rule=OnePositionAtATime)

    def InitialPositionConstraint(model,s):
        return model.Y[params.StartingPoint,tInit,s] == 1
    model.InitialPositionConstraint = pyo.Constraint(model.S,rule=InitialPositionConstraint)

    def FinalPositionConstraint(model,s):
        return model.Y[params.EndingPoint,tEnd,s] == 1
    model.FinalPositionConstraint = pyo.Constraint(model.S,rule=FinalPositionConstraint)

    def NonanticipativityConstraint_X(model,r,s):
        return model.X[r,0,1] == model.X[r,0,s]
    model.NonanticipativityConstraint_X = pyo.Constraint(model.R * model.S_NON_1,rule=NonanticipativityConstraint_X)

    def NonanticipativityConstraint_Y(model,p,s):
        return model.Y[p,0,1] == model.Y[p,0,s]
    model.NonanticipativityConstraint_Y = pyo.Constraint(model.P * model.S_NON_1,rule=NonanticipativityConstraint_Y)

    ### Step 6: Define Objective
    model.Obj = pyo.Objective(expr=sum(params.pi[s] * sum(params.delta[r][t][s] * model.X[r,t,s] for r in model.R for t in model.T) for s in model.S),sense=pyo.minimize)

    return model

In [36]:
def ExecuteOptimization(model:pyo.ConcreteModel,tee=False):
    """
    A function to execute the optimization of a pyomo model.

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model you'd like to optimize
    tee: boolean (optional, Default=False)
        An indication of whether or not you'd like to print the solver's progress as it solves.

    Returns
    -------
    None
    """

    #Sometimes there are lots of settings you want to set or special functions you want to call to record the output of this "solve call". That's why I normally have a dedicated function for this. But I suppose it's less necessary for this simple problem.
    solver = pyo.SolverFactory("scip")
    solver.solve(model,tee=tee)

In [37]:
def ExtractResults(model:pyo.ConcreteModel):
    """
    A function to extract the results of a Route Planning Problem with Multiple Time Periods and Multiple Scenarios model.

    IMPORTANT NOTE: The goal of this style of "Stochastic" model is not to determine ALL of the decisions that should be made right off the bat.
        Instead, we just want to know what we should do RIGHT NOW. 
        Then, once the uncertainty about the future becomes more clear, we'll re-optimize from that time point (with new available decisions and new probabilities) to determine the best choice for that moment.
        Thus, the most important result to extract here is what we should do in the very first time period.


    Parameters
    ----------
    model: pyo.ConcreteModel
        The model from which you'd like to extract the results

    Returns
    -------
    traveledPaths: list
        A list of the paths that are to be traveled under the optimal solution
    """
    t = 0
    s = 1 #likewise, since we have non-anticipativity constraints for every variable in the first time period, we only need to look at one of the scenarios.

    instructions = []
    found = False
    for r in model.R:
        Xval = pyo.value(model.X[r,t,s])
        if Xval >= 0.5:
            found = True
            instructions.append(f"At t = {t}, travel down path {r}")
    if not found:
        instructions.append(f"At t = {t}, Stay put at A")

    instructions.append("Then, if Scenario 1 ends up happening...")

    #Gather instructions if scenario 1 ends up happening.
    s = 1
    for t in model.T:
        if t == 0:
            continue
        for r in model.R:
            Xval = pyo.value(model.X[r,t,s])
            if Xval >= 0.5:
                instructions.append(f"\tAt t = {t}, travel down path {r}")

    instructions.append("Or, if Scenario 2 ends up happening...")
    s = 2
    for t in model.T:
        if t == 0:
            continue
        for r in model.R:
            Xval = pyo.value(model.X[r,t,s])
            if Xval >= 0.5:
                instructions.append(f"\tAt t = {t}, travel down path {r}")

    return instructions

In [38]:
def main():
    params = Parameters()
    model = AssembleModel(params)
    ExecuteOptimization(model)
    results = ExtractResults(model)

    for step in results:
        print(step)

main()

At t = 0, travel down path AB
Then, if Scenario 1 ends up happening...
	At t = 1, travel down path BC
	At t = 2, travel down path CD
Or, if Scenario 2 ends up happening...
	At t = 1, travel down path BD


# Route Planning with EZ Pass option


$\delta_{r,t}$

|$r$|$t=0$|$t=1$|$t=2$|$t=3$|
|-|-|-|-|-|
|$AB$|$15|$10|$8|$8|
|$AC$|$5|$5|$9|$10|
|$BC$|$4|$10|$13|$13|
|$BD$|$2|$2|$6|$5|
|$CD$|$10|$15|$4|$7|

$\delta^{EZ}_{r,t}$

|$r$|$t=0$|$t=1$|$t=2$|$t=3$|
|-|-|-|-|-|
|$AB$|$0|$0|$0|$0|
|$AC$|$5|$5|$9|$10|
|$BC$|$4|$10|$13|$13|
|$BD$|$2|$2|$6|$5|
|$CD$|$0|$0|$0|$0|

Price of the EZ Pass: $\rho^{EZ} = \$12$

$$\min_{...} \rho^{EZ}Z + \sum_{r \in \textbf{R}} \sum_{t \in \textbf{T}} C_{r,t} $$
$$s.t.\ \ \ Y_{p,t+1} - Y_{p,t} = \sum_{r\in\textbf{R}^+_p} X_{r,t} - \sum_{r\in\textbf{R}^-_p} X_{r,t}$$
$$\phantom\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall p \in \textbf{P}, t \in \textbf{T} \neq 3$$
$$\sum_{r \in \textbf{R}} X_{r,t} \leq 1 \ \ \ \forall t \in \textbf{T}$$
$$\sum_{p \in \textbf{P}} Y_{p,t} = 1 \ \ \ \forall t \in \textbf{T}$$
$$Y_{A,0} = 1$$
$$Y_{3,D} = 1$$
$$C^Z_{r,t} = \begin{cases}\delta^{EZ}_{r,t} & if\ Z \\ \delta_{r,t} & else\end{cases} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} = \begin{cases}C^Z_{r,t} & if\  X_{r,t} \\ 0.0 & else\end{cases} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$

$$X_{r,t}, Y_{p,t} \in \{0,1\}$$

Notice how the trick is to break the definition of the cost of traveling down a path, $C_{r,t}$, into two "if" statements.

Now we can use the definitions and caveats found on slides 16 and 17 to re-formulate these if statements to a form that fits within the math modeling language:

We'll start with $C^Z_{r,t}$:
$$C^Z_{r,t} = C^{Z,Z}_{r,t} + C^{Z,not\ Z}_{r,t}$$
$$C^{Z,Z}_{r,t} = \begin{cases} \delta^{EZ}_{r,t} & if\ Z \\ 0.0 & else\end{cases} = \delta^{EZ}_{r,t} Z $$
$$C^{Z,not\ Z}_{r,t} = \begin{cases} \delta_{r,t} & if\ 1-Z \\ 0.0 & else\end{cases} = \delta_{r,t} (1-Z) $$

Notice how these are already linear since they are both a parameter (constant) times a variable. So we don't actually need to break them down even further.
$$C^Z_{r,t} = \delta^{EZ}_{r,t} Z + \delta_{r,t} (1-Z) $$

Next we'll handle $C_{r,t}$:
$$C_{r,t} = C^{X}_{r,t} + C^{not\ X}_{r,t}$$
$$C^{Z}_{r,t} = \begin{cases} C^{Z}_{r,t} & if\ X_{r,t} \\ 0.0 & else\end{cases} = C^{Z}_{r,t} X_{r,t} $$
$$C^{not\ X}_{r,t} = \begin{cases} 0.0 & if\ 1-Z \\ 0.0 & else\end{cases} = 0.0$$

Unfortunately, the term $C^{Z}_{r,t} X_{r,t}$ is not linear, so we need to break this one down even further. In order to do that, we'll need to define the $\beta^{MAX}$ and $\beta^{MIN}$ terms.

Here, we need the maximum and minimum possible values of $C^{Z}_{r,t}$. In our case, since $\delta^{EZ}_{r,t}$ is allways less than $\delta_{r,t}$, we easily know that the maximum value is just $\delta^{EZ}_{r,t}$ and the minimum value is just $\delta_{r,t}$.

Thus, using the definition from the right-hand side of slide 16,

$$C_{r,t} \leq \delta^{EZ}_{r,t} X_{r,t} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \leq C^{Z}_{r,t} + \delta_{r,t} (X_{r,t}-1) \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \geq \delta_{r,t} X_{r,t} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \geq C^{Z}_{r,t} + \delta^{EZ}_{r,t} (X_{r,t}-1) \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$

But here, remember that under now regions of the feasible space are we EVER going to be maximizing this cost. We will always be minimizing this cost. So we can drop the "less than" constraints.

In then end, we simplify 
$$C^Z_{r,t} = \begin{cases}\delta^{EZ}_{r,t} & if\ Z \\ \delta_{r,t} & else\end{cases} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} = \begin{cases}C^Z_{r,t} & if\  X_{r,t} \\ 0.0 & else\end{cases} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$

all the way down to:
$$C^Z_{r,t} = \delta^{EZ}_{r,t} Z + \delta_{r,t} (1-Z)  \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \geq \delta_{r,t} X_{r,t} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \geq C^{Z}_{r,t} + \delta^{EZ}_{r,t} (X_{r,t}-1) \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$

THUS, our final formulation is:
$$\min_{...} \rho^{EZ}Z + \sum_{r \in \textbf{R}} \sum_{t \in \textbf{T}} C_{r,t} $$
$$s.t.\ \ \ Y_{p,t+1} - Y_{p,t} = \sum_{r\in\textbf{R}^+_p} X_{r,t} - \sum_{r\in\textbf{R}^-_p} X_{r,t}$$
$$\phantom\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall p \in \textbf{P}, t \in \textbf{T} \neq 3$$
$$\sum_{r \in \textbf{R}} X_{r,t} \leq 1 \ \ \ \forall t \in \textbf{T}$$
$$\sum_{p \in \textbf{P}} Y_{p,t} = 1 \ \ \ \forall t \in \textbf{T}$$
$$Y_{A,0} = 1$$
$$Y_{3,D} = 1$$
$$C^Z_{r,t} = \delta^{EZ}_{r,t} Z + \delta_{r,t} (1-Z)  \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \geq \delta_{r,t} X_{r,t} \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$
$$C_{r,t} \geq C^{Z}_{r,t} + \delta^{EZ}_{r,t} (X_{r,t}-1) \ \ \ \forall r \in \textbf{R}, t \in \textbf{T}$$

$$X_{r,t}, Y_{p,t} \in \{0,1\}$$

Now let's code this in.

In [39]:
class Parameters:
    """
    A class to house all the parameters relevant to the Route Planning with Multiple Time Periods and EZ pass option Example Problem
    """
    def __init__(self):
        #Specify all parameters here making sure to say "self." before each one.
        self.Points = ["A","B","C","D"]
        self.Connections = ["AB","AC","BC","BD","CD"]
        self.StartingPoint = "A"
        self.EndingPoint = "D"

        self.TimePoints = [0,1,2,3]

        self.delta = {
            "AB": {0: 15., 1: 10., 2: 8. , 3: 8. },
            "AC": {0: 5. , 1: 5. , 2: 9. , 3: 10.},
            "BC": {0: 4. , 1: 10., 2: 13., 3: 13.},
            "BD": {0: 2. , 1: 2. , 2: 6. , 3: 5. },
            "CD": {0: 10., 1: 15., 2: 4. , 3: 7. }
        }

        self.delta_EZ = {
            "AB": {0: 0. , 1: 0. , 2: 0. , 3: 0. },
            "AC": {0: 5. , 1: 5. , 2: 9. , 3: 10.},
            "BC": {0: 4. , 1: 10., 2: 13., 3: 13.},
            "BD": {0: 2. , 1: 2. , 2: 6. , 3: 5. },
            "CD": {0: 0. , 1: 0. , 2: 0. , 3: 0. }
        }


        self.gamma_EZ = 12.

In [40]:
from PyomoTools import LoadIndexedSet

def AssembleModel(params:Parameters):
    """
    A function to generate an instance of the Pyomo model for the Route Planning with Multiple Time Periods Example Problem.

    Parameters
    ----------
    params: Parameters
        The Parameters object containing the parameters relevant to this instance.

    Returns
    -------
    model: pyo.ConcreteModel
        The Pyomo model for this instance.
    """

    ### STEP 1: Define the Pyomo model
    model = pyo.ConcreteModel()

    ### STEP 2: Define sets
    model.P = pyo.Set(initialize=params.Points)
    model.R = pyo.Set(initialize=params.Connections)
    
    R_plus = {p: [r for r in model.R if p == r[1]] for p in model.P}
    R_minus = {p: [r for r in model.R if p == r[0]] for p in model.P}
    LoadIndexedSet(model,"R_plus",R_plus) #This is a function I created to automate the process of adding subsets into a Pyomo model.
    LoadIndexedSet(model,"R_minus",R_minus)
    
    model.T = pyo.Set(initialize=params.TimePoints)
    tInit = min(params.TimePoints)
    tEnd = max(params.TimePoints)
    model.T_NON_END = pyo.Set(initialize=[t for t in model.T if t != tEnd])

    ### Step 3: Define parameters
    #This is already done in the Parameters class

    ### Step 4: Define variables
    model.X = pyo.Var(model.R * model.T,domain=pyo.Binary)

    model.Y = pyo.Var(model.P * model.T,domain=pyo.Binary)

    model.C_Z = pyo.Var(model.R * model.T, domain=pyo.Reals)

    model.C = pyo.Var(model.R * model.T, domain=pyo.Reals)

    model.Z = pyo.Var(domain=pyo.Binary)

    ### Step 5: Define constraints
    def TransitionDefinitionConstraint(model,p,t):
        return model.Y[p,t+1] - model.Y[p,t] == sum(model.X[r,t] for r in model.R_plus[p]) - sum(model.X[r,t] for r in model.R_minus[p])
    model.TransitionDefinitionConstraint = pyo.Constraint(model.P * model.T_NON_END,rule=TransitionDefinitionConstraint)

    def NoMoreThanOneTransitionAtATime(model,t):
        return sum(model.X[r,t] for r in model.R) <= 1
    model.NoMoreThanOneTransitionAtATime = pyo.Constraint(model.T, rule=NoMoreThanOneTransitionAtATime)

    def OnePositionAtATime(model,t):
        return sum(model.Y[p,t] for p in model.P) == 1
    model.OnePositionAtATime = pyo.Constraint(model.T, rule=OnePositionAtATime)

    def InitialPositionConstraint(model):
        return model.Y[params.StartingPoint,tInit] == 1
    model.InitialPositionConstraint = pyo.Constraint(rule=InitialPositionConstraint)

    def FinalPositionConstraint(model):
        return model.Y[params.EndingPoint,tEnd] == 1
    model.FinalPositionConstraint = pyo.Constraint(rule=FinalPositionConstraint)

    def C_Z_Definition(model,r,t):
        return model.C_Z[r,t] == params.delta_EZ[r][t] * model.Z + params.delta[r][t] * (1-model.Z)
    model.C_Z_Definition = pyo.Constraint(model.R * model.T, rule=C_Z_Definition)

    def C_Definition_1(model,r,t):
        return model.C[r,t] >= params.delta[r][t] * model.X[r,t]
    model.C_Definition_1 = pyo.Constraint(model.R * model.T,rule=C_Definition_1)

    def C_Definition_2(model,r,t):
        return model.C[r,t] >= model.C_Z[r,t] + params.delta_EZ[r][t] * (model.X[r,t]-1)
    model.C_Definition_2 = pyo.Constraint(model.R * model.T,rule=C_Definition_2)

    ### Step 6: Define Objective
    model.Obj = pyo.Objective(expr=sum(model.C[r,t] for r in model.R for t in model.T)+params.gamma_EZ*model.Z,sense=pyo.minimize)

    return model

In [41]:
def ExecuteOptimization(model:pyo.ConcreteModel,tee=False):
    """
    A function to execute the optimization of a pyomo model.

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model you'd like to optimize
    tee: boolean (optional, Default=False)
        An indication of whether or not you'd like to print the solver's progress as it solves.

    Returns
    -------
    None
    """

    #Sometimes there are lots of settings you want to set or special functions you want to call to record the output of this "solve call". That's why I normally have a dedicated function for this. But I suppose it's less necessary for this simple problem.
    solver = pyo.SolverFactory("scip")
    solver.solve(model,tee=tee)

In [42]:
def ExtractResults(model:pyo.ConcreteModel):
    """
    A function to extract the results of a Route Planning Problem with Multiple Time Periods and EZ Pass model.

    The point of this problem is just to determine whether or not I should buy an EZ pass.

    I could extract more information. But I'll just return this one result.

    Parameters
    ----------
    model: pyo.ConcreteModel
        The model from which you'd like to extract the results

    Returns
    -------
    verdict: bool
        An indication of whether or not I should purchase an EZ pass.
    """
    return pyo.value(model.Z) >= 0.5

In [43]:
def main():
    params = Parameters()
    model = AssembleModel(params)
    ExecuteOptimization(model)
    result = ExtractResults(model)

    if result:
        print("I should buy an EZ pass!")
    else:
        print("I should NOT buy an EZ pass.")

main()

I should buy an EZ pass!


If you want to play around with the model, try to determine the threshold cost of an EZ pass $\gamma^{EZ}$ where it would no longer be worth it.